## Sentiment Analysis

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import zeros
import nltk
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Embedding, LSTM
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

nltk.download('stopwords')
from nltk.corpus import stopwords

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data
df = pd.read_csv("./data/training.1600000.processed.noemoticon.csv", encoding = 'latin', header = None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
# change columns names
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
# drop useless columns
df = df.drop(['id', 'date', 'flag', 'user'], axis = 1)
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
# count dataset samples to know how many of each class we have
df.target.value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [6]:
def clean_text(text):
    text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words]
    
    return " ".join(tokens)
    
df.text = df.text.apply(lambda x: clean_text(x))

In [7]:
df.head()

,target,text
0,0,awww bummer shoulda got david carr third day
1,0,upset update facebook texting might cry result...
2,0,dived many times ball managed save 50 rest go ...
3,0,whole body feels itchy like fire
4,0,behaving mad see


In [8]:
train_size = 0.8

train_data, test_data = train_test_split(df, test_size = 1 - train_size, random_state = 0, stratify = df.target)

In [9]:
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))

Train data size:  1280000
Test data size:  320000


In [10]:
print("Train data distr: \n", train_data.target.value_counts())
print("Test data distr: \n", test_data.target.value_counts())

Train data distr: 
 4    640000
0    640000
Name: target, dtype: int64
Test data distr: 
 4    160000
0    160000
Name: target, dtype: int64


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.text)

In [12]:
# padding

max_length = max([len(s.split()) for s in train_data.text])

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text),
                        maxlen = max_length)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text),
                       maxlen = max_length)

In [14]:
# label encoding

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train_data.target.to_list())

y_train = encoder.transform(train_data.target.to_list())
y_test = encoder.transform(test_data.target.to_list())

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [15]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape:  (1280000, 50)
x_test shape:  (320000, 50)
y_train shape: (1280000, 1)
y_test shape: (320000, 1)


In [17]:
# embedding layer

# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf-8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, embedding_dim))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

# contains the index for each word
vocab = tokenizer.word_index
# total number of words in our vocabulary, plus one for unknown words
vocab_size = len(tokenizer.word_index) + 1
# embedding dimensions
embedding_dim = 300

# load embedding from file
raw_embedding = load_embedding('./data/glove/glove.6B.300d.txt')
# get vectors in the right order
embedding_matrix = get_weight_matrix(raw_embedding, vocab)

# create the embedding layer
embedding_layer = Embedding(vocab_size, 
                            embedding_dim, 
                            weights = [embedding_matrix], 
                            input_length = max_length, 
                            trainable = False)

In [19]:
# define model

from keras.layers.core import Dropout

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

print(model.summary())

model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           87214200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 87,374,701
Trainable params: 160,501
Non-trainable params: 87,214,200
_________________________________________________________________
None


In [20]:
# train model

batch_s = 1024
num_epochs = 10

ReduceLROnPlateau = ReduceLROnPlateau(factor = 0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

history = model.fit(x_train, y_train, batch_size = batch_s, epochs = num_epochs,
                    validation_split = 0.1, callbacks = [ReduceLROnPlateau])

Train on 1152000 samples, validate on 128000 samples
Epoch 1/10
1152000/1152000 [==============================] - 126s 109us/step - loss: 0.5429 - accuracy: 0.7205 - val_loss: 0.4857 - val_accuracy: 0.7637
Epoch 2/10
1152000/1152000 [==============================] - 123s 107us/step - loss: 0.5084 - accuracy: 0.7469 - val_loss: 0.4740 - val_accuracy: 0.7719
Epoch 3/10
1152000/1152000 [==============================] - 124s 108us/step - loss: 0.4986 - accuracy: 0.7536 - val_loss: 0.4670 - val_accuracy: 0.7765
Epoch 4/10
1152000/1152000 [==============================] - 125s 109us/step - loss: 0.4930 - accuracy: 0.7578 - val_loss: 0.4624 - val_accuracy: 0.7793
Epoch 5/10
1152000/1152000 [==============================] - 125s 108us/step - loss: 0.4886 - accuracy: 0.7611 - val_loss: 0.4597 - val_accuracy: 0.7811
Epoch 6/10
1152000/1152000 [==============================] - 125s 108us/step - loss: 0.4857 - accuracy: 0.7624 - val_loss: 0.4580 - val_accuracy: 0.7829
Epoch 7/10
1152000/1152

In [21]:
model.save('third_model.h5')

In [22]:
# evaluate model

score = model.evaluate(x_test, y_test, batch_size = batch_s)
print("Test accuracy:", score[1])
print("Test loss:", score[0])

320000/320000 [==============================] - 14s 42us/step
Test accuracy: 0.7828812599182129
Test loss: 0.4584811236381531


In [23]:
def decode_pred(prediction):
    return 0 if prediction < 0.5 else 1

In [29]:
data_string = clean_text("@taylorswift i love it! it's amazing but im a little bit sad")
data_string = pad_sequences(tokenizer.texts_to_sequences([data_string]),
                        maxlen = max_length)

prediction = model.predict([data_string])[0]
label = decode_pred(prediction)

print("Prediction: {} Score: {}".format(prediction, label))

Prediction: [0.04597246] Score: 0
